In [1]:
import numpy as np
import pandas as pd

import sqlite3

In [2]:
with sqlite3.connect('logs/logs.db') as conn:
    logs = pd.read_sql(
    """
    SELECT * FROM logs WHERE datetime BETWEEN '2018-01-01' AND '2019-01-01';
    """, conn, parse_dates = ['datetime'], index_col='datetime'
    )
    
logs.head()

,source_ip,username,success,failure_reason
datetime,,,,
2018-01-01 00:06:19.353126,223.178.55.3,djones,1,None
2018-01-01 00:09:07.147971,223.178.55.3,djones,1,None
2018-01-01 01:08:08.610041,6.252.142.27,asmith,1,None
2018-01-01 02:37:50.329298,124.178.25.98,akim,1,None
2018-01-01 02:45:20.382080,98.43.141.103,akim,1,None


In [3]:
def get_X(log, day):
    """
    Get data we can use for the X
    
    Parameters:
        - log: The logs dataframe
        - day: A day or single value we can use as a datetime index slice
    
    Returns: 
        A pandas DataFrame
    """
    return pd.get_dummies(log[day].assign(
        failures=lambda x:  1 - x.success
    ).query('failures > 0').resample('1min').agg(
        {'username':'nunique', 'failures': 'sum'}
    ).dropna().rename(
        columns={'username':'usernames_with_failures'}
    ).assign(
        day_of_week=lambda x: x.index.dayofweek, 
        hour=lambda x: x.index.hour
    ).drop(columns=['failures']), columns=['day_of_week', 'hour'])

In [4]:
x = get_X(logs, '2018-01')
x.head()

,usernames_with_failures,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,hour_0,hour_1,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 16:01:00,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2018-01-01 16:02:00,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2018-01-01 16:03:00,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2018-01-01 16:04:00,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2018-01-01 16:05:00,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [5]:
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [6]:
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('svm', OneClassSVM(random_state=0))
]).fit(x)

c:\users\korze\desktop\python\hands-on\book_venv\book_env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\korze\desktop\python\hands-on\book_venv\book_env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
c:\users\korze\desktop\python\hands-on\book_venv\book_env\lib\site-packages\sklearn\svm\classes.py:1177: DeprecationWarning: The random_state parameter is deprecated and will be removed in version 0.22.
  " be removed in version 0.22.", DeprecationWarning)


In [7]:
preds = pipeline.predict(x)

c:\users\korze\desktop\python\hands-on\book_venv\book_env\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [11]:
pd.Series(np.where(preds == -1, 'outlier', 'inlier')).value_counts()

outlier    22823
inlier     18794
dtype: int64